## Day 47 Lecture 1 Assignment

In this assignment, we will apply k-means clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here
nba = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
print(nba.shape)
nba.head()

(2141, 30)


,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9


The goal is to cluster these player-seasons to identify potential player "archetypes".

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [6]:
# answer goes here
#less_20g = nba[nba['GS'] < 20]
#less_10mp = nba[nba['MP'] < 10.0]

#cleaned data with players meeting criteria
nba_clean = nba[(nba['MP'] >= 10.0) & (nba['GS'] >= 20)]

In [7]:
nba_clean

,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
7,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
10,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
12,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2122,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
2125,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
2126,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [8]:
# answer goes here
#ast, trb, stl, blk, tov, pts adding free throws(ft)
X = nba_clean[['AST', 'TRB', 'STL', 'BLK', 'TOV', 'PTS', 'FT']]
X 

,AST,TRB,STL,BLK,TOV,PTS,FT
3,1.6,9.5,1.5,1.0,1.7,13.9,1.8
4,2.2,7.3,0.9,0.8,1.5,8.9,2.0
7,2.4,9.2,0.5,1.3,1.8,21.3,4.3
10,1.4,8.4,0.5,1.5,1.3,10.9,2.5
12,1.3,7.5,0.8,0.4,0.9,9.4,1.9
...,...,...,...,...,...,...,...
2122,5.8,2.9,0.9,0.2,2.3,14.1,2.8
2125,2.5,2.5,0.9,0.3,1.6,15.3,5.3
2126,1.4,6.4,0.7,1.0,0.8,11.7,1.5
2138,1.8,9.0,1.5,0.5,1.9,15.1,1.4


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [9]:
# answer goes here
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale

,AST,TRB,STL,BLK,TOV,PTS,FT
0,-0.617816,1.612909,1.393220,0.824001,0.023511,0.114638,-0.281459
1,-0.330886,0.768627,-0.050271,0.420162,-0.218831,-0.744985,-0.155144
2,-0.235242,1.497780,-1.012598,1.429758,0.144681,1.386880,1.297476
3,-0.713459,1.190768,-1.012598,1.833596,-0.461172,-0.401136,0.160643
4,-0.761281,0.845380,-0.290853,-0.387514,-0.945854,-0.659023,-0.218302
...,...,...,...,...,...,...,...
799,1.390695,-0.919937,-0.050271,-0.791352,0.750535,0.149023,0.350115
800,-0.187421,-1.073443,-0.050271,-0.589433,-0.097660,0.355332,1.929050
801,-0.713459,0.423239,-0.531435,0.824001,-1.067025,-0.263596,-0.470931
802,-0.522172,1.421027,1.393220,-0.185595,0.265852,0.320947,-0.534089


Run K-means clustering with K = 3 and print out the resulting centroids. When printing the centroids, transform the scaled centroids back into their corresponding unscaled values. What "archetypes" do you see?

In [13]:
# answer goes here
kmeans = KMeans(n_clusters=3)

#fit model
#corresponding clusters 
X['cluster'] = kmeans.fit_predict(X_scale)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,AST,TRB,STL,BLK,TOV,PTS,FT,cluster
3,1.6,9.5,1.5,1.0,1.7,13.9,1.8,2
4,2.2,7.3,0.9,0.8,1.5,8.9,2.0,1
7,2.4,9.2,0.5,1.3,1.8,21.3,4.3,2
10,1.4,8.4,0.5,1.5,1.3,10.9,2.5,2
12,1.3,7.5,0.8,0.4,0.9,9.4,1.9,1
...,...,...,...,...,...,...,...,...
2122,5.8,2.9,0.9,0.2,2.3,14.1,2.8,0
2125,2.5,2.5,0.9,0.3,1.6,15.3,5.3,1
2126,1.4,6.4,0.7,1.0,0.8,11.7,1.5,1
2138,1.8,9.0,1.5,0.5,1.9,15.1,1.4,2


In [14]:
#inversing scaled transformation
clusters = scale.inverse_transform(kmeans.cluster_centers_)

In [16]:
#making into dataframe to see values and correlations
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

,AST,TRB,STL,BLK,TOV,PTS,FT
0,5.829143,5.410286,1.343429,0.508000,2.805714,20.065143,4.113714
1,2.097737,4.181481,0.803704,0.406996,1.247737,10.300412,1.431276
2,1.996503,8.950350,0.802098,1.323077,1.774825,14.839860,2.727273


It seems that 3 clusters is not the best k for this dataset as cluster labeled 0 has the highest points in 5 of the 7 categories and cluster labeled 2 holds the last remaining 2 highest points. Cluster labeld 1 does not have any high or closely high data points. 

Experiment with different values of K. Do any further interesting archetypes come out?

In [20]:
#k=2
kmeans = KMeans(n_clusters=2)

#fit model
kmeans.fit_predict(X_scale)

#clusters
clusters = scale.inverse_transform(kmeans.cluster_centers_)

#dataframe
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

,AST,TRB,STL,BLK,TOV,PTS,FT
0,2.032601,4.957432,0.790541,0.555743,1.305743,10.826014,1.576182
1,5.291509,6.245755,1.284906,0.692925,2.727358,19.955189,4.115094


In [17]:
# answer goes here
#k=4
kmeans = KMeans(n_clusters=4)

#fit model
kmeans.fit_predict(X_scale)

#clusters
clusters = scale.inverse_transform(kmeans.cluster_centers_)

#dataframe
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

,AST,TRB,STL,BLK,TOV,PTS,FT
0,6.105385,5.803077,1.407692,0.544615,2.950000,21.728462,4.680769
1,1.391941,4.319048,0.637729,0.492308,0.973626,8.205128,1.091575
2,2.017293,9.148872,0.778947,1.339098,1.796241,14.955639,2.786466
3,3.295149,4.136567,1.043657,0.345522,1.727612,13.379478,1.971642


In [18]:
#k=5
kmeans = KMeans(n_clusters=5)

#fit model
kmeans.fit_predict(X_scale)

#clusters
clusters = scale.inverse_transform(kmeans.cluster_centers_)

#dataframe
clusters = pd.DataFrame(clusters, columns=X_scale.columns)
clusters.style.background_gradient()

,AST,TRB,STL,BLK,TOV,PTS,FT
0,1.526582,3.521097,0.664979,0.362869,0.954430,8.213502,1.024895
1,1.764804,7.231844,0.716201,0.873743,1.425698,11.982123,1.949721
2,3.746818,3.938182,1.100455,0.335000,1.876364,13.860455,2.087727
3,2.300000,10.589286,0.930357,1.778571,2.158929,17.891071,3.735714
4,6.199107,5.986607,1.432143,0.537500,3.000893,22.293750,4.866964


When I changed k to 4 and 5, the clusters showed a drastic difference in where the highest points were located. When n_clusters=4, cluster labeled 0 had the most of the highest points (5/7) similarily when n_clusters=3. When n_clusters=5, cluster labeled 4 held the most of the highest points instead of cluster labeled 0. 

When n_clusters=2, the clusters show a distinct seperation between the 2 clusters where one (cluster labeled 1) holds all the highest points and the other (cluster labeled 0) holds none of the highest points.

The 5 statistics that always seem to be in the same cluster are Assists, Steals, Turn overs, points, and free throws. Grouping these statistics together make sense as usually the team/player who has the highest points in one usually has the highest points in the others which would make them the better player. 
(except for turnovers those don't make sense to me to be in the same cluster.)